Scores a LLM trained on a calibrated dataset with DPO using alpaca_eval prompts and evaluation criteria

In [ ]:
!pip install alpaca_eval

In [ ]:
!pip install datasets
import datasets
eval_set = datasets.load_dataset("tatsu-lab/alpaca_eval", "alpaca_eval")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from alpaca_eval import evaluate
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Uses alpaca eval to compare the output of the trained model to the output of the original model. Edit model name and saved model paths for a given model.

In [ ]:
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

model_name = ""
base_tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForCausalLM.from_pretrained(model_name).cuda()

trained_model_path = ""
trained_tokenizer = AutoTokenizer.from_pretrained(trained_model_path, local_files_only=True)
trained_model = AutoModelForCausalLM.from_pretrained(trained_model_path, local_files_only=True).cpu()
base_model.eval()
trained_model.eval()

eval_path_base = ""
eval_path_trained = ""

prompts = eval_set

trained_responses = []
base_responses = []
i=0
for prompt in prompts:
    base_inputs = base_tokenizer(prompt, return_tensors="pt", truncation=True).to(base_model.device)
    base_output = base_model.generate(**base_inputs, max_new_tokens=200)
    base_response = base_tokenizer.decode(base_output[0], skip_special_tokens=True)
    base_responses.append(base_response)

    trained_inputs = trained_tokenizer(prompt, return_tensors="pt", truncation=True).to(trained_model.device)
    trained_output = trained_model.generate(**trained_inputs, max_new_tokens=200)
    trained_response = trained_tokenizer.decode(trained_output[0], skip_special_tokens=True)
    trained_responses.append(trained_response)

    print(f"{i}: {trained_response}")
    i+=1

import pandas as pd
trained_df = pd.DataFrame(trained_responses, columns=["response"], index=False)
base_df = pd.DataFrame(base_responses, columns=["response"], index=False)

trained_df.to_csv(eval_path_trained)
base_df.to_csv(eval_path_base)

In [ ]:
results = evaluate(
    model_outputs=model_a_generations,
    reference_outputs=model_b_generations,
    model_name="Model A",
    reference_name="Model B"
)

print(results)